# 导入工具包

In [20]:
# opencv-python
import cv2

# mediapipe人工智能工具包
import mediapipe as mp

# 进度条库
from tqdm import tqdm

# 时间库
import time

# 导入模型

In [21]:
# 导入人脸检测模型
mp_face_detection = mp.solutions.face_detection
model = mp_face_detection.FaceDetection(   
        min_detection_confidence=0.5, # 置信度阈值，过滤掉小于置信度的预测框
        model_selection=0,            # 选择模型，0适用于人脸距离镜头近（2米以内），1适用于人脸距离镜头远（5米以内）
)

In [22]:
# 导入可视化函数和可视化样式
mp_drawing = mp.solutions.drawing_utils
# 关键点样式
keypoint_style = mp_drawing.DrawingSpec(thickness=5, circle_radius=3, color=(0,255,0))
# 人脸预测框样式
bbox_style = mp_drawing.DrawingSpec(thickness=5, circle_radius=3, color=(255,0,0))

# 处理单帧的函数

In [23]:
# 处理帧函数
def process_frame(img):
    
    # 记录该帧开始处理的时间
    start_time = time.time()
    
    # 获取图像宽高
    h, w = img.shape[0], img.shape[1]
    
    # BGR转RGB
    img_RGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # 将RGB图像输入模型，获取预测结果
    results = model.process(img_RGB)
    
    if results.detections: # 如果检测出人脸
        
        # 可视化人脸框和人脸关键点
        for detection in results.detections:
            mp_drawing.draw_detection(img, 
                                      detection,
                                      # bbox_drawing_spec=bbox_style,
                                      keypoint_drawing_spec=keypoint_style
                                     )

        # 用不同颜色可视化人脸框
        face_num = len(results.detections) # 检测出的人脸个数
        # 设置人脸框颜色列表，BGR
        color_list = [(66,77,229),(223,155,6),(94,218,121),(180,187,28),(77,169,10),(1,240,255)]
        for i in range(face_num): # 遍历每一个人脸框
            top_X = int(results.detections[i].location_data.relative_bounding_box.xmin * w)
            top_Y = int(results.detections[i].location_data.relative_bounding_box.ymin * h)
            bbox_width = int(results.detections[i].location_data.relative_bounding_box.width * w)
            bbox_height = int(results.detections[i].location_data.relative_bounding_box.height * h)
            # 绘制人脸矩形框，左上角坐标，右下角坐标，颜色，线宽
            img = cv2.rectangle(img, (top_X, top_Y),(top_X+bbox_width, top_Y+bbox_height),color_list[i%face_num],15)
    else: # 未检测出人脸
        scaler = 1
        img = cv2.putText(img, 'No Face Detected', (25 * scaler, 100 * scaler), cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (255, 0, 255), 2 * scaler)
    
    # 记录该帧处理完毕的时间
    end_time = time.time()
    # 计算每秒处理图像帧数FPS
    FPS = 1/(end_time - start_time)
    
    scaler = 1
    # 在图像上写FPS数值，参数依次为：图片，添加的文字，左上角坐标，字体，字体大小，颜色，字体粗细
    img = cv2.putText(img, 'FPS  '+str(int(FPS)), (25 * scaler, 50 * scaler), cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (255, 0, 255), 2 * scaler)

    return img

# 视频逐帧处理（模板）

In [24]:
# 视频逐帧处理代码模板
# 不需修改任何代码，只需定义process_frame函数即可


def generate_video(input_path='./videos/three-hands.mp4'):
    filehead = input_path.split('/')[-1]
    output_path = "out-" + filehead
    
    print('视频开始处理',input_path)
    
    # 获取视频总帧数
    cap = cv2.VideoCapture(input_path)
    frame_count = 0
    while(cap.isOpened()):
        success, frame = cap.read()
        frame_count += 1
        if not success:
            break
    cap.release()
    print('视频总帧数为',frame_count)
    
    # cv2.namedWindow('Crack Detection and Measurement Video Processing')
    cap = cv2.VideoCapture(input_path)
    frame_size = (cap.get(cv2.CAP_PROP_FRAME_WIDTH), cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))
    # fourcc = cv2.VideoWriter_fourcc(*'XVID')
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)

    out = cv2.VideoWriter(output_path, fourcc, fps, (int(frame_size[0]), int(frame_size[1])))
    
    # 进度条绑定视频总帧数
    with tqdm(total=frame_count-1) as pbar:
        try:
            while(cap.isOpened()):
                success, frame = cap.read()
                if not success:
                    break

                # 处理帧
                # frame_path = './temp_frame.png'
                # cv2.imwrite(frame_path, frame)
                try:
                    frame = process_frame(frame)
                except:
                    print('error')
                    pass
                
                if success == True:
                    # cv2.imshow('Video Processing', frame)
                    out.write(frame)

                    # 进度条更新一帧
                    pbar.update(1)

                # if cv2.waitKey(1) & 0xFF == ord('q'):
                    # break
        except:
            print('中途中断')
            pass

    cv2.destroyAllWindows()
    out.release()
    cap.release()
    print('视频已保存', output_path)

In [25]:
generate_video(input_path='videos/test_video.mp4')

视频开始处理 videos/test_video.mp4


  0%|          | 1/415 [00:00<00:55,  7.52it/s]

视频总帧数为 416


100%|██████████| 415/415 [00:11<00:00, 36.53it/s]

视频已保存 out-test_video.mp4
